In [3]:
import pandas as pd

# general cleaning data

In [ ]:
# nutrition and obesity cleaning

nutrition_and_obesity_data =  pd.read_csv('../data/provided-data-raw/Nutrition_Physical_Activity_and_Obesity_Data.csv')

# remove obviously useless or redundant data
nutrition_and_obesity_data = nutrition_and_obesity_data.dropna(subset=['Data_Value'])
nutrition_and_obesity_data = nutrition_and_obesity_data.drop(['YearEnd', 'Data_Value_Unit', 'Data_Value_Type', 'Data_Value_Alt',
                                  'Data_Value_Footnote_Symbol', 'Data_Value_Footnote'], axis=1)
nutrition_and_obesity_data = nutrition_and_obesity_data.rename(columns={'YearStart': 'Year'})

# need to figure out how we want to further wrangle the data
nutrition_and_obesity_data.to_csv('../data/provided-data-clean/nutrition-and-obesity-cleaned.csv')

In [8]:
# merged data cleaning
data = pd.read_csv('../data/data-state-for-processing/merged_dataset.csv')
# drop if contains unnamed
data = data.loc[:, ~data.columns.str.contains('^Unnamed')]

# drop specific columns by name
columns_to_drop = [
    'Housing_Units',
    'Population_Half',
    'Low_Income_Half',
    'Vehicle_Half',
    'Population_1',
    'Low_Income_1',
    'Vehicle_1'
]
data = data.drop(columns=columns_to_drop)

# by year
years_to_drop = ['2010', '2011', '2013', '2015']
columns_to_drop = [col for col in data.columns if any(year in col for year in years_to_drop)]
data = data.drop(columns=columns_to_drop)

print(data)

# Save the updated DataFrame to a new csv
data.to_csv('../data/data-state-for-processing/merged_dataset_cleaned.csv', index=False)



   State  Urban_Percentage  Total_Population  Median_Family_Income  \
0     AL         54.668930         4779736.0          59814.492716   
1     AK         50.299401          710231.0          87459.460606   
2     AZ         85.000000         6392017.0          69963.005972   
3     AR         48.833819         2915918.0          56778.258443   
4     CA         91.637587        37253956.0          88387.665785   
5     CO         78.663446         5029196.0          86548.935772   
6     CT         85.265700         3574097.0         100079.497567   
7     DE         79.439252          897934.0          80919.785047   
8     DC        100.000000          601723.0         119155.768786   
9     FL         86.728838        18801310.0          68329.029908   
10    GA         67.399080         9687653.0          68388.554756   
11    HI         84.112150         1360301.0          93827.152104   
12    ID         57.046980         1567582.0          64366.379195   
13    IL         84.

# extract from nutrition and obesity

In [14]:
# extract specifically obesity data for students
nutrition_and_obesity_data =  pd.read_csv('../data/data-clean/nutrition-and-obesity-cleaned.csv')
obesity_data = nutrition_and_obesity_data[nutrition_and_obesity_data['Question'] == 
                                          'Percent of adults aged 18 years and older who have obesity']

# obesity_state_2011 = obesity_data[obesity_data['Year'] == 2011].groupby('LocationAbbr').agg({'Data_Value': 'mean'}).reset_index()
# obesity_state_2011 = obesity_state_2011.rename(columns={'Data_Value': 'Mean_Obesity_Pct'})

# obesity_state_2015 = obesity_data[obesity_data['Year'] == 2015].groupby('LocationAbbr').agg({'Data_Value': 'mean'}).reset_index()
# obesity_state_2015 = obesity_state_2015.rename(columns={'Data_Value': 'Mean_Obesity_Pct'})

# obesity_state_2011.to_csv('../data/provided-data-clean/obesity-state-2011-cleaned.csv')
# obesity_state_2015.to_csv('../data/provided-data-clean/obesity-state-2015-cleaned.csv')

obesity_state_2019 = obesity_data[obesity_data['Year'] == 2019].groupby('LocationAbbr').agg({'Data_Value': 'mean'}).reset_index()
obesity_state_2019 = obesity_state_2019.rename(columns={'Data_Value': 'Mean_Obesity_Pct'})

obesity_state_2019.to_csv('../data/data-state-for-processing/obesity-state-2019-cleaned.csv')



IndentationError: unexpected indent (190309218.py, line 22)

In [16]:
fruit_data = nutrition_and_obesity_data[nutrition_and_obesity_data['Question'] ==
                                            'Percent of adults who report consuming fruit less than one time daily']

veggie_data = nutrition_and_obesity_data[nutrition_and_obesity_data['Question'] ==
                                            'Percent of adults who report consuming vegetables less than one time daily']

fruit_state_2019 = fruit_data[fruit_data['Year'] == 2019].groupby('LocationAbbr').agg({'Data_Value': 'mean'}).reset_index()
fruit_state_2019 = fruit_state_2019.rename(columns={'Data_Value': 'Mean_Fruit_Consump_Pct'})

veggie_state_2019 = veggie_data[veggie_data['Year'] == 2019].groupby('LocationAbbr').agg({'Data_Value': 'mean'}).reset_index()
veggie_state_2019 = veggie_state_2019.rename(columns={'Data_Value': 'Mean_Veggie_Consump_Pct'})

fruit_state_2019.to_csv('../data/data-state-for-processing/fruit-state-2019-cleaned.csv')
veggie_state_2019.to_csv('../data/data-state-for-processing/veggie-state-2019-cleaned.csv')

In [6]:
nutrition_and_obesity_data =  pd.read_csv('../data/data-clean/nutrition-and-obesity-cleaned.csv')

# sugary drink data
soda_data = nutrition_and_obesity_data[nutrition_and_obesity_data['Question'] == 
                                        'Percent of students in grades 9-12 who drank regular soda/pop at least one time per day']

soda_state_2019 = soda_data[soda_data['Year'] == 2019].groupby('LocationAbbr').agg({'Data_Value': 'mean'}).reset_index()
soda_state_2019 = soda_state_2019.rename(columns={'Data_Value': 'Soda_Consump_Pct'})
soda_state_2019.rename(columns={'LocationAbbr': 'State'}, inplace=True)
print(soda_state_2019)

# put into merged data
merged_data = pd.read_csv('../data/data-state-for-processing/merged_dataset.csv')
updated_data = pd.merge(merged_data, soda_state_2019, on='State', how='left')
print(updated_data)

/var/folders/xr/0vzl322108z0xw4wh8532jj80000gn/T/ipykernel_21409/2840244166.py:1: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  nutrition_and_obesity_data =  pd.read_csv('../data/data-clean/nutrition-and-obesity-cleaned.csv')


   State  Soda_Consump_Pct
0     AK         13.475000
1     AL         21.330000
2     AR         22.490000
3     AZ         13.820000
4     CA         13.620000
5     CT         13.118182
6     DC         13.583333
7     FL         16.208333
8     GU         14.709091
9     HI         10.425000
10    IA         17.300000
11    ID         11.555556
12    IL         13.190909
13    KS         16.388889
14    KY         24.310000
15    LA         19.670000
16    MA         11.345455
17    MD         12.021429
18    MI         14.391667
19    MO         21.120000
20    MS         24.670000
21    MT         13.154545
22    NC         18.525000
23    ND         16.190000
24    NE         16.200000
25    NH         11.015385
26    NJ          9.818182
27    NM         18.900000
28    NV         10.863636
29    NY         12.553846
30    OH         15.710000
31    OK         22.133333
32    PA         12.291667
33    PR         25.100000
34    SC         19.700000
35    SD         14.754545
3

# food environ atlas

In [6]:
# clean food evironment atlas

food_environ_data = pd.read_csv('../data/other-data-raw/food-environment-atlas-raw/state_county_data_raw.csv')
variable_names = pd.read_csv('../data/other-data-raw/food-environment-atlas-raw/variable_list_pruned.csv')

valid_variable_codes = variable_names['Variable Code'].tolist()
print(valid_variable_codes)

# filter the rows based on the variable codes
filtered_data = food_environ_data[food_environ_data['Variable_Code'].isin(valid_variable_codes)]
# drop the FIPS column
filtered_data = filtered_data.drop(columns=['FIPS'])

filtered_data.to_csv('../data/other-data-clean/food_environ_county_data_clean.csv', index=False)

['LACCESS_POP10', 'LACCESS_POP15', 'LACCESS_LOWI10', 'LACCESS_LOWI15', 'LACCESS_HHNV10', 'LACCESS_HHNV15', 'LACCESS_SNAP15', 'LACCESS_CHILD10', 'LACCESS_CHILD15', 'LACCESS_SENIORS10', 'LACCESS_SENIORS15', 'LACCESS_WHITE15', 'LACCESS_BLACK15', 'LACCESS_HISP15', 'LACCESS_NHASIAN15', 'LACCESS_NHNA15', 'LACCESS_NHPI15', 'LACCESS_MULTIR15', 'GROC11', 'GROC16', 'GROCPTH11', 'GROCPTH16', 'SUPERC11', 'SUPERC16', 'SUPERCPTH11', 'SUPERCPTH16', 'CONVS11', 'CONVS16', 'CONVSPTH11', 'CONVSPTH16', 'SPECS11', 'SPECS16', 'SPECSPTH11', 'SPECSPTH16', 'SNAPS12', 'SNAPS17', 'SNAPSPTH12', 'SNAPSPTH17', 'WICS11', 'WICS16', 'WICSPTH11', 'WICSPTH16', 'FFR11', 'FFR16', 'FFRPTH11', 'FFRPTH16', 'FSR11', 'FSR16', 'FSRPTH11', 'FSRPTH16', 'PC_FFRSALES07', 'PC_FFRSALES12', 'PC_FSRSALES07', 'PC_FSRSALES12', 'FOOD_BANKS18', 'FMRKT13', 'FMRKT18', 'FMRKTPTH13', 'FMRKTPTH18', 'FMRKT_FRVEG18', 'FMRKT_ANMLPROD18', 'FMRKT_BAKED18', 'PCT_DIABETES_ADULTS08', 'PCT_DIABETES_ADULTS13', 'PCT_OBESE_ADULTS12', 'PCT_OBESE_ADULTS17', 

In [10]:
# extract state data from county data
cleaned_county_data = pd.read_csv('../data/other-data-clean/food_environ_county_data_clean.csv')
variable_names = pd.read_csv('../data/other-data-raw/food-environment-atlas-raw/variable_list_primary.csv')

# only include count variables
valid_variable_codes = variable_names[variable_names['Units'].str.contains('Count', na=False)]['Variable Code'].tolist()
filtered_data = cleaned_county_data[cleaned_county_data['Variable_Code'].isin(valid_variable_codes)]

# group by 'State' and 'Variable_Code' columns and sum the 'Value' column
state_data = filtered_data.groupby(['State', 'Variable_Code'], as_index=False)['Value'].sum()

# add variable name column to improve readability
variable_code_to_name = variable_names.set_index('Variable Code')['Variable Name'].to_dict()
state_data['Variable Name'] = state_data['Variable_Code'].map(variable_code_to_name)
state_data = state_data[['State', 'Variable Name', 'Variable_Code', 'Value']]

state_data.to_csv('../data/other-data-clean/food_environ_state_data_clean.csv', index=False)

# not needed

In [9]:
# extract median household income

spreadsheet = pd.read_csv('../data/data-raw/acs_5yr_est_selected_economic_characteristics_2010-2022.csv')
print(spreadsheet)

spreadsheet['Label (Grouping)'] = spreadsheet['Label (Grouping)'].str.strip()

# Filter the rows where "Label (Grouping)" is "Median household income (dollars)" and "Year" is 2019
filtered_data = spreadsheet[(spreadsheet['Label (Grouping)'] == 'Median household income (dollars)') & (spreadsheet['Year'] == 2019)]
print(filtered_data)
filtered_data = filtered_data[['State', 'Estimate', 'Year']]

# Save the filtered data to a new CSV file
filtered_data.to_csv('../data/data-state-for-processing/median_household_income_2019.csv', index=False)

                              Label (Grouping)  \
0                            EMPLOYMENT STATUS   
1                 Population 16 years and over   
2                               In labor force   
3                         Civilian labor force   
4                                     Employed   
...                                        ...   
98015                        18 years and over   
98016                           18 to 64 years   
98017                        65 years and over   
98018                       People in families   
98019  Unrelated individuals 15 years and over   

                                                Category        State  \
0                                                 Header      Alabama   
1                                      EMPLOYMENT STATUS      Alabama   
2                                      EMPLOYMENT STATUS      Alabama   
3                                      EMPLOYMENT STATUS      Alabama   
4                                 